In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: | ^C
failed

CondaError: KeyboardInterrupt

     |████████████████████████████████| 92kB 14.9MB/s eta 0:00:01
Libraries imported.


In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
!pip install folium
import folium # map rendering library

print('Libraries imported.')

# Capstone Project

#### Applied Data Science Capstone by IBM/Coursera

# Location for a burger joint

## 1. Introduction/Business Problem

Find a good location to start a business is a challenge problem. 
A young man wants start his first business. He makes very tasty hamburgers and decided to open a burger joint. He lives between two universities and thinks that locations where there are many students are good to sell hamburgers because students like it. He wants to know what is the best location to open his business, that is, near what university and in what region near the university.

In this project we will help this young choose a location to sell hamburgers in Campinas, State of São Paulo, Brazil. We will take into the number of burger joints that already exist in the neighborhood of universities, if necessary we will include other categories of restaurants to improve our analysis. We will use Foursquare api to find them.

We will make Exploratory Data Analysis and use a clustering algorithm to  detect locations that are not already crowded with burger joints and 
we will choose a locations as close to university as possible.

## 2. Data

To solve our problem we need:

- University locations;

- Burger joint and others restaurants locations. 


The university name's were provided by young that will start the business. And we use Google maps to get the adress.

### 2.1 University Locations

#### 2.1.1 Adress

##### 2.1.1.1 UNICAMP

**Name:** Universidade Estadual de Campinas

**Adress:** Cidade Universitária Zeferino Vaz - Barão Geraldo, Campinas - SP, 13083-970

**Site:** https://www.unicamp.br/unicamp/mapas


##### 2.1.1.2 MACKENZIE

**Name:** Universidade Presbiteriana Mackenzie, Campus Campinas

**Adress:** Av. Brasil, 1220 - Jardim Guanabara, Campinas - SP, 13073-148

**Site:** https://www.mackenzie.br/universidade/campinas/

#### 2.1.2 Google Maps url

We use the Google Maps url to take latitude and longitude values of Universities. The latitude and longitude are at the end of url.

##### 2.1.2.1 Google Maps - UNICAMP

https://www.google.com.br/maps/place/UNICAMP+Universidade+Estadual+de+Campinas/@-22.8176241,-47.0681567,15z/data=!4m5!3m4!1s0x94c8c6b005d24db5:0xc6db750ecf04d796!8m2!3d-22.8184393!4d-47.0647206?hl=pt-BR



In [3]:
latitude_u, longitude_u =[-22.8184393,-47.0647206]

##### 2.1.2.2 Google Maps - Mackenzie

https://www.google.com.br/maps/place/Av.+Brasil,+1220+-+Jardim+Guanabara,+Campinas+-+SP,+13073-148/@-22.8854167,-47.0706735,17z/data=!3m1!4b1!4m5!3m4!1s0x94c8c61ad1cc9435:0x94dba75d07160912!8m2!3d-22.8854217!4d-47.0684848?hl=pt-BR


In [4]:
latitude_m, longitude_m = [-22.8854217,-47.0684848]

#### 2.1.3 City of Campinas with the two universities

Using geopy to get Campinas' geographical coordinates

In [9]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("Campinas - SP")
print(location.address)
latitude = location.latitude
longitude = location.longitude

Campinas, Região Imediata de Campinas, Região Metropolitana de Campinas, Região Intermediária de Campinas, São Paulo, Região Sudeste, Brasil


In [10]:
# create map of Campinas using latitude and longitude values
map_campinas = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map

# Unicamp
folium.Marker(
    location=[latitude_u, longitude_u],
    popup='Unicamp',
    icon=folium.Icon(color='red')
).add_to(map_campinas)

# Mackenzie
folium.Marker(
    location=[latitude_m, longitude_m],
    popup='Mackenzie',
    icon=folium.Icon(color='green')
).add_to(map_campinas)

map_campinas

### 2.2 Restaurant locations

Next, we are going to start utilizing the Foursquare API to get the burger joint locations. 



#### 2.2.1 Define Foursquare Credentials and Version

In [11]:
# The code was removed by Watson Studio for sharing.

In [12]:
# The content of above cell
# CLIENT_ID = 'XXXXXXX' # your Foursquare ID
# CLIENT_SECRET = 'XXXXXXXXXXX' # your Foursquare Client Secret
# VERSION = '20180605' # Foursquare API version

#### 2.2.2 Search for burger near universities

##### 2.2.2.1 Unicamp

In [13]:
# Unicamp
search_query = 'burger'
radius = 1000
LIMIT = 50
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_u, longitude_u, VERSION, search_query, radius, LIMIT)



In [14]:
results = requests.get(url).json()

In [15]:
results

{'meta': {'code': 200, 'requestId': '5e83a8ab77af03001b3cb721'},
 'response': {'venues': []}}

The search for burger returned zero results so we decided include other food categories (Burger Joint,
Bakery,
Fast Food Restaurant,
Food Stand,
Food Truck,
Hot Dog Joint,
Pastelaria,
Pizza Place,
Sandwich Place,
Snack Place). We take the categories from [Foursquare Documentation](https://developer.foursquare.com/docs/build-with-foursquare/categories/).

In [16]:
## The categories included and their ids are:

#Burger Joint  
'4bf58dd8d48988d16c941735',
# Bakery
'4bf58dd8d48988d16a941735',
# Fast Food Restaurant
'4bf58dd8d48988d16e941735',
# Food Stand
'56aa371be4b08b9a8d57350b',
# Food Truck
'4bf58dd8d48988d1cb941735',
# Hot Dog Joint
'4bf58dd8d48988d16f941735',
# Pastelaria
'5294cbda3cf9994f4e043a63',
# Pizza Place
'4bf58dd8d48988d1ca941735',
# Sandwich Place
'4bf58dd8d48988d1c5941735',
# Snack Place
'4bf58dd8d48988d1c7941735'

'4bf58dd8d48988d1c7941735'

#### 2.2.3 Search for restaurants

In [17]:
# A comma separated list of categories
cat_ids = '4bf58dd8d48988d16c941735,4bf58dd8d48988d16a941735,4bf58dd8d48988d16e941735,56aa371be4b08b9a8d57350b,4bf58dd8d48988d1cb941735,4bf58dd8d48988d16f941735,5294cbda3cf9994f4e043a63,4bf58dd8d48988d1ca941735,4bf58dd8d48988d1c5941735,4bf58dd8d48988d1c7941735'


In [18]:
# search parameters
CATEGORYID = cat_ids
radius = 1000
LIMIT = 50

##### 2.2.3.1 Unicamp

In [19]:
# unicamp
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_u, longitude_u, VERSION, CATEGORYID, radius, LIMIT)


In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e83a8c2b57e88001b885647'},
 'response': {'venues': [{'id': '53106897498e0a8b5508fcc6',
    'name': 'Cantina da FEC',
    'location': {'lat': -22.816377430666996,
     'lng': -47.062061677426634,
     'labeledLatLngs': [{'label': 'display',
       'lat': -22.816377430666996,
       'lng': -47.062061677426634}],
     'distance': 356,
     'cc': 'BR',
     'country': 'Brasil',
     'formattedAddress': ['Brasil']},
    'categories': [{'id': '4bf58dd8d48988d1c7941735',
      'name': 'Snack Place',
      'pluralName': 'Snack Places',
      'shortName': 'Snacks',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/snacks_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1585686654',
    'hasPerk': False},
   {'id': '5068d796e4b02e9a846e8a83',
    'name': 'Pizzaria Piramide',
    'location': {'lat': -22.82454092929293,
     'lng': -47.0722022020202,
     'labeledLatLngs': [{'label': 'display',
       'lat': -22.82

In [21]:
venues = results['response']['venues']

nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.columns

Index(['categories', 'hasPerk', 'id', 'location.address', 'location.cc',
       'location.city', 'location.country', 'location.crossStreet',
       'location.distance', 'location.formattedAddress',
       'location.labeledLatLngs', 'location.lat', 'location.lng',
       'location.neighborhood', 'location.postalCode', 'location.state',
       'name', 'referralId'],
      dtype='object')

In [22]:
# filter columns
filtered_columns = ['name', 'categories', 'location.lat',
       'location.lng', 'location.distance']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,name,categories,location.lat,location.lng,location.distance
0,Cantina da FEC,"[{'id': '4bf58dd8d48988d1c7941735', 'name': 'S...",-22.816377,-47.062062,356
1,Pizzaria Piramide,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",-22.824541,-47.072202,1025
2,Lanchonete do IEL,"[{'id': '4bf58dd8d48988d1c7941735', 'name': 'S...",-22.815227,-47.070109,658
3,Delão Lanches,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",-22.815784,-47.073780,975
4,Burguer King Shopping Dom Pedro,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",-22.813271,-47.062553,616
5,Quiosque Do Açaí,"[{'id': '4bf58dd8d48988d1c7941735', 'name': 'S...",-22.820020,-47.071950,762
6,Veg Wrap,"[{'id': '56aa371be4b08b9a8d57350b', 'name': 'F...",-22.817171,-47.070477,607
7,Rango Da Une,"[{'id': '56aa371be4b08b9a8d57350b', 'name': 'F...",-22.817047,-47.069927,556
8,King Stoned,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",-22.826250,-47.072360,1170
9,Pastelaria Ideal,"[{'id': '4bf58dd8d48988d1c7941735', 'name': 'S...",-22.822477,-47.070224,721


We need to extract category from category column. Lets take a look at first cell

In [23]:
nearby_venues.categories[0]

[{'id': '4bf58dd8d48988d1c7941735',
  'name': 'Snack Place',
  'pluralName': 'Snack Places',
  'shortName': 'Snacks',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/snacks_',
   'suffix': '.png'},
  'primary': True}]

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,distance
0,Cantina da FEC,Snack Place,-22.816377,-47.062062,356
1,Pizzaria Piramide,Pizza Place,-22.824541,-47.072202,1025
2,Lanchonete do IEL,Snack Place,-22.815227,-47.070109,658
3,Delão Lanches,Food Truck,-22.815784,-47.073780,975
4,Burguer King Shopping Dom Pedro,Fast Food Restaurant,-22.813271,-47.062553,616


In [26]:
venues_unicamp = nearby_venues.copy()
print("Number of restaurants in Unicamp's neighborhood:{}".format(len(venues_unicamp)))


Number of restaurants in Unicamp's neighborhood:27


##### 2.2.3.2 Mackenzie

In [27]:
# mackenzie
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_m, longitude_m, VERSION, CATEGORYID, radius, LIMIT)

In [28]:
results = requests.get(url).json()

venues = results['response']['venues']

nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['name', 'categories', 'location.lat',
       'location.lng', 'location.distance']
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [29]:
# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,distance
0,Panificadora Crisabel,Bakery,-22.883295,-47.068249,237
1,We Can Veg It,Food Truck,-22.890164,-47.067267,542
2,Ki-Dog Brasil,Hot Dog Joint,-22.880988,-47.072124,618
3,Subway,Sandwich Place,-22.881397,-47.072166,585
4,Pizzaria Nonna Cecília,Pizza Place,-22.882965,-47.068306,274


In [30]:
venues_mackenzie = nearby_venues.copy()
print("Number of restaurants in Mackenzie's neighborhood:{}".format(len(venues_mackenzie)))

Number of restaurants in Mackenzie's neighborhood:49


#### 2.2.4 Maps: Universities and restaurants

In [31]:
map_unicamp = folium.Map(location=[latitude_u,longitude_u], zoom_start=14)
folium.Marker([latitude_u,longitude_u], popup='Unicamp').add_to(map_unicamp)
for lat, lon in zip(venues_unicamp['lat'],venues_unicamp['lng']):
    folium.CircleMarker([lat, lon], radius=3, fill=True, fill_opacity=1).add_to(map_unicamp)
map_unicamp

In [32]:
map_mackenzie = folium.Map(location=[latitude_m,longitude_m], zoom_start=14)
folium.Marker([latitude_m,longitude_m], popup='Mackenzie').add_to(map_mackenzie)
for lat, lon in zip(venues_mackenzie['lat'],venues_mackenzie['lng']):
    folium.CircleMarker([lat, lon], radius=3, fill=True, fill_opacity=1).add_to(map_mackenzie)
map_mackenzie

#### 2.3 Final Data

In this section we get the location of two universities and locations of some categories of restaurants in their neighboorhood. Our firs attempt was search for 'burger' but the request return 0 venues. So we decided include other categories we think students like to. Now we have two dataframes, one to each university. Each row of dataframes contains: name of restaurant,	category(Burger Joint,
Bakery,
Fast Food Restaurant,
Food Stand,
Food Truck,
Hot Dog Joint,
Pastelaria,
Pizza Place,
Sandwich Place,
Snack Place), geographical coordinates and	distance to university.